# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,stanley,54.8680,-1.6985,16.55,92,40,1.03,GB,1691636418
1,1,enewetak,11.3474,162.3373,28.71,76,87,6.63,MH,1691636418
2,2,rio grande,-32.0350,-52.0986,15.53,96,100,5.07,BR,1691636419
3,3,la passe,45.5549,-0.8967,21.13,76,8,3.15,FR,1691636419
4,4,port-aux-francais,-49.3500,70.2167,3.11,95,100,10.89,TF,1691636419


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
humidity_map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height=500,
    size = "Humidity",
    color = "City",
    alpha = 0.8)

# Display the map
humidity_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values

# Narrow Down To Favorable Temperature Range
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] <27) & (city_data_df["Max Temp"] >21)]

# Narrow Daown To Favorable Wind Speed
city_data_df = city_data_df.loc[(city_data_df["Wind Speed"] <4.5)]

# Narrow Down Based On Cloudy Conditions (Cloudiness)
city_data_df = city_data_df.loc[(city_data_df["Cloudiness"] <1)]

# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
6,6,bamut,43.1599,45.1973,23.73,62,0,1.82,RU,1691636419
41,41,casper,42.8666,-106.3131,22.33,47,0,3.60,US,1691636281
102,102,gillette,44.2911,-105.5022,24.51,41,0,2.06,US,1691636171
117,117,pirallahi,40.4723,50.3330,26.08,49,0,3.33,AZ,1691636449
160,160,tughyl,47.7252,84.2063,24.45,47,0,2.63,KZ,1691636456
164,164,waddan,29.1614,16.1390,22.94,67,0,1.80,LY,1691636456
221,221,argo,37.6333,22.7333,22.66,75,0,1.19,GR,1691636379
230,230,gargalianoi,37.0667,21.6333,22.42,76,0,0.82,GR,1691636471
275,275,remire-montjoly,4.9167,-52.2667,26.02,89,0,2.06,GF,1691636241
297,297,uberlandia,-18.9186,-48.2772,21.56,46,0,3.60,BR,1691636485


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
6,bamut,RU,43.1599,45.1973,62,
41,casper,US,42.8666,-106.3131,47,
102,gillette,US,44.2911,-105.5022,41,
117,pirallahi,AZ,40.4723,50.3330,49,
160,tughyl,KZ,47.7252,84.2063,47,
164,waddan,LY,29.1614,16.1390,67,
221,argo,GR,37.6333,22.7333,75,
230,gargalianoi,GR,37.0667,21.6333,76,
275,remire-montjoly,GF,4.9167,-52.2667,89,
297,uberlandia,BR,-18.9186,-48.2772,46,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories" : "accommodation.hotel",
          "apiKey" : geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bamut - nearest hotel: No hotel found
casper - nearest hotel: Days Inn by Wyndham Casper
gillette - nearest hotel: Montgomery Hotel
pirallahi - nearest hotel: No hotel found
tughyl - nearest hotel: No hotel found
waddan - nearest hotel: No hotel found
argo - nearest hotel: Apollon Hotel
gargalianoi - nearest hotel: Messinian Horizons
remire-montjoly - nearest hotel: Complexe Belova
uberlandia - nearest hotel: Hotel Presidente
cruz do espirito santo - nearest hotel: No hotel found
santa cruz - nearest hotel: Aviador Hotel Boutique
ilha solteira - nearest hotel: Hotel Urubupungá
nili - nearest hotel: No hotel found
quartu sant'elena - nearest hotel: Hotel Italia
arish - nearest hotel: سينا
ardestan - nearest hotel: Tourism Hotel
mancio lima - nearest hotel: No hotel found
ierapetra - nearest hotel: No hotel found
pierre - nearest hotel: Quality Inn
emerald - nearest hotel: Emerald Central Hotel
north platte - nearest hotel: Howard Johnson Inn - North Platte


,City,Country,Lat,Lng,Humidity,Hotel Name
6,bamut,RU,43.1599,45.1973,62,No hotel found
41,casper,US,42.8666,-106.3131,47,Days Inn by Wyndham Casper
102,gillette,US,44.2911,-105.5022,41,Montgomery Hotel
117,pirallahi,AZ,40.4723,50.3330,49,No hotel found
160,tughyl,KZ,47.7252,84.2063,47,No hotel found
164,waddan,LY,29.1614,16.1390,67,No hotel found
221,argo,GR,37.6333,22.7333,75,Apollon Hotel
230,gargalianoi,GR,37.0667,21.6333,76,Messinian Horizons
275,remire-montjoly,GF,4.9167,-52.2667,89,Complexe Belova
297,uberlandia,BR,-18.9186,-48.2772,46,Hotel Presidente


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height=500,
    size = "Humidity",
    color = "City",
    alpha = 0.8,
    hover_cols = ["Hotel Name", "Country"])

# Display the map
hotel_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)